In [1]:
!pip uninstall -y transformers accelerate evaluate rouge_score
!pip install transformers accelerate evaluate rouge_score
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d72dd8218a123ed9f4f1697555fa3739eabf3dff86789fba495617f23807

In [2]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 21.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=77adfa2919a399efcd2f10e24eef79cb18b6bd6d3b014ae2fc9638374819789b
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [3]:
import wandb
wandb.init(project ="EmailTask")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

import evaluate
import numpy as np
import pandas as pd
import tensorflow as tf

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [6]:
train_file_path = "/content/drive/MyDrive/AESLC-master/emailSubject.txt"
eval_file_path = "/content/drive/MyDrive/AESLC-master/EmailSubjectEval.txt"
model_name = 'distilgpt2'
rouge = evaluate.load('rouge')
sacrebleu = evaluate.load("sacrebleu")
output_dir = '/content/AESLC-master/result'
overwrite_output_dir = False
per_device_train_batch_size = 1
num_train_epochs = 3
save_steps = 10000

In [7]:
list_of_email = []
list_of_subject = []
list_of_ann0 = []
list_of_ann1 = []
list_of_ann2 = []
list_of_references = []
with open(eval_file_path, "r") as file:
    for line in file:
        file_content = line.replace("\t", " ").replace("\n", " ").split("Subject :")
        word = file_content[0]
#         print(line)
        if len(word.split()) <= 400 :
            try:
              subject = file_content[1].split("ann0 :")[0]
              ann0 = file_content[1].split("ann0 :")[1].split("ann1 :")[0]
              ann1 = file_content[1].split("ann0 :")[1].split("ann1 :")[1].split("ann2 :")[0]
              ann2 = file_content[1].split("ann0 :")[1].split("ann1 :")[1].split("ann2 :")[1]
              list_of_email.append(file_content[0])
              list_of_references.append([subject, ann0, ann1, ann2])
            except :
              print()

In [12]:
def load_dataset(file_path, tokenizer):
    dataset = LineByLineTextDataset(
                tokenizer=tokenizer,
                file_path=file_path,
                block_size=512
    )

    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm
    )

    return data_collator


def postprocess_text(preds, labels):
    # TODO: Separate only the subject from string
    # Ensure that for preds, you have a list of only the generated subject parts
    # For labels, it should be a list of list of only the reference subjects
    # NO OTHER CONTENT: EMAIL / SEPARATORS SHOULD BE OUTPUT AFTER POSTPROCESSING

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    # print('logits:', logits.shape)
    pred_ids = torch.argmax(logits, dim=-1)
    # print('pred_ids:', pred_ids.shape)

    return pred_ids, labels


def compute_metrics(eval_preds):
    i = 0
    size = len(list_of_email)
    list_of_prediction = []
    while i < size:
        print(i)
        print(list_of_references[i])
        inputs = tokenizer(list_of_email[i] + ' Subject : ')
        device = torch.device("cuda:0")
        model.to(device)

        inputs['input_ids'] = inputs['input_ids']
        outputs = model.generate(inputs['input_ids'], max_new_tokens = 5, do_sample = True, top_k = 30, top_p = 0.95)
        prediction = prediction.split('Subject : ')[1]
        list_of_prediction.append(prediction)
        print(prediction)
        i = i + 1

    result = rouge.compute(predictions=list_of_prediction, references=list_of_references)
    result_sacrebleu = sacrebleu.compute(predictions=list_of_prediction, references=list_of_references, lowercase = True)
    wandb.log({
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(result_sacrebleu["score"], 4)})
    return {
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(result_sacrebleu["score"], 4)
    }


tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model = GPT2LMHeadModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.save_pretrained(output_dir)

train_dataset = load_dataset(train_file_path, tokenizer)
eval_dataset = load_dataset(eval_file_path, tokenizer)
data_collator = load_data_collator(tokenizer)

tokenizer.save_pretrained(output_dir)

training_args = TrainingArguments(
          output_dir=output_dir,
          evaluation_strategy = "epoch",
          #eval_steps = 1000,
          learning_rate=1e-5,
          save_strategy = "epoch",
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
        #   per_device_eval_batch_size=1,
          num_train_epochs=num_train_epochs
      )

trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          preprocess_logits_for_metrics=preprocess_logits_for_metrics,
          compute_metrics=compute_metrics
)
with wandb.init(project="emailTask"):
    trainer.train()
    trainer.save_model()

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0
['  Service  ', '  need help with blackberry service  ', '  trouble getting service  ', '  no service in the woodlands  ']


train/epoch,▁█
train/global_step,▁█
train/learning_rate,█▁
train/loss,█▁
train/epoch,0.68
train/global_step,1000
train/learning_rate,1e-05
train/loss,4.1127


AttributeError: ignored

In [ ]:
list_of_email = []
list_of_subject = []
list_of_ann0 = []
list_of_ann1 = []
list_of_ann2 = []
list_of_prediction = []
list_of_references = []
file = open("/content/drive/MyDrive/AESLC-master/EmailSubjectTest.txt", "r", encoding = 'utf-8')
count = 0
for line in file:
            count += 1
            print(count)
            file_content = line.split("Subject : ")
            print(file_content)
            word = file_content[0]
            if len(word.split()) <= 100 :
              list_of_email.append(' '.join(word.split()))
              list_of_subject.append(file_content[1].split("ann0 : ")[0])
              print(file_content[1].split("ann0 : ")[0])
              list_of_ann0.append(file_content[1].split("ann0 : ")[1].split("ann1 : ")[0])
              print(file_content[1].split("ann0 : ")[1].split("ann1 : ")[0])
              list_of_ann1.append(file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[0])
              print(file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[0])
              list_of_ann2.append(file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[1])
              print(file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[1])
              list_of_references.append([file_content[1].split("ann0 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[1]])
              #print([file_content[1].split("ann0 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[0], file_content[1].split("ann0 : ")[1].split("ann1 : ")[1].split("ann2 : ")[1]])

Streaming output truncated to the last 5000 lines.
first draft attached, please review and advise 
first draft revisions needed

951
["Email : PLEASE READ THIS IMPORTANT INFORMATION CONCERNING THE ENRON STOCK FUND IN THE ENRON CORP. SAVINGS AND ESOP PLANS  On Friday, January 11, trading of Enron stock was suspended on the New York Stock Exchange (NYSE).All Savings and ESOP Plan transactions, including 401(k) payroll contributions,  involving this fund were placed in a pending status until trading resumed.As announced by Enron's Office of the Chair on Wednesday, January 16, Enron stock would now be traded Over The Counter (OTC).Upon receiving this information, work began immediately to develop a process that allows for OTC trades in the plans.On Thursday, January 17, Enron initiated a process to facilitate OTC trades of Enron stock inside the 401(k) and ESOP and all pending transactions were processed using Thursday's closing price.Now that trading has resumed, we urge you to review you

In [ ]:
def test_loss():
    i = 0
    size = len(list_of_email)
    list_of_prediction = []
    while i < size:
              print(i)
              print(list_of_references[i])
              inputs = tokenizer(list_of_email[i] + ' Subject : ', return_tensors="pt")
              inputs['input_ids'] = inputs['input_ids'].cpu()  # Move input tensor to CPU if necessary
              device = torch.device("cuda:0")  # Specify the CUDA device
              model.to(device)  # Move the model to the CUDA device

              # Move the input tensor to the CUDA device
              inputs['input_ids'] = inputs['input_ids'].to(device)
              outputs = model.generate(inputs['input_ids'], max_new_tokens=5, do_sample=True, top_k=30, top_p=0.95)
              prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
              # Generate outputs using the model on the CUDA device
              print(prediction)
              prediction = prediction.split('Subject :')[1]
              list_of_prediction.append(prediction)
              print(prediction)
              i = i + 1
    result = rouge.compute(predictions=list_of_prediction, references=list_of_references)
    results_sacrebleu = sacrebleu.compute(predictions=list_of_prediction, references=list_of_references, lowercase = True)
    #results_meteor = meteor.compute(predictions=decoded_preds, references=decoded_labels)
    return {
        "R1": round(result["rouge1"], 4),
        "R2": round(result["rouge2"], 4),
        "RL": round(result["rougeL"], 4),
        "RLsum": round(result["rougeLsum"], 4),
        "bleu": round(results_sacrebleu["score"], 4)
    }

In [ ]:
test_loss_metrics = test_loss()

In [ ]:
test_loss_metrics